# Table Manager

**Objective:** Create tables with default values, and test unique contraints.


In [1]:
import os, sys, json
import pandas as PD
import numpy as NP

In [2]:
MODULE_PATH = os.path.dirname(os.getcwd())
DATA_PATH = os.path.join(MODULE_PATH, 'data')
LOGIC_PATH = os.path.join(MODULE_PATH, 'logic')
sys.path.append(LOGIC_PATH)
print(MODULE_PATH)

C:\Documents\Code\OpenStack\Current\MediaManager\codebase\pictures_manager


In [3]:
import sqlalchemy as SQL
import database_helper as DB
print([x for x in dir(DB) if '_' not in x and len(x) > 8])
dbPath = os.path.join(DATA_PATH, 'fiction.db')
meta, engine = DB.openConnection(dbPath)

['openConnection']
sqlite:///C:\Documents\Code\OpenStack\Current\MediaManager\codebase\pictures_manager\data\fiction.db


In [4]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import IPython.display as DS 
from ipywidgets import IntProgress

### Create Tables

In [5]:
# get existing table names
# reflect the meta again to refresh any deleted tables

meta.clear()
meta.reflect()

existingTables = list(meta.tables.keys())
print(existingTables)

[]


In [6]:
# create table if it doesn't already exist
# default parameter doesn't work with inserts
# server_default requires parameter value as string

tableName = 'ArtistTracker'

if tableName not in existingTables:
    setsTb = SQL.Table(tableName, meta, 
        SQL.Column('id', SQL.Integer, primary_key = True), 
        SQL.Column('collectionId', SQL.String, server_default='no-id'), 
        SQL.Column('deviantId', SQL.String, server_default='no-id'),
        SQL.Column('artistType', SQL.String),                             # canvas, cosplay, pinup
        SQL.Column('status', SQL.String, server_default='to-do'),         # to do, last download date, skip
                                                                          # photographer?, inactive
        SQL.Column('favourites', SQL.Boolean, server_default='0'),
        SQL.Column('collected', SQL.Integer),
        SQL.Column('downloaded', SQL.Integer),
        SQL.UniqueConstraint('collectionId', 'deviantId', name='id_unique')
    )
    meta.create_all(engine)
    print(f"table {tableName} created")

else:
    print(f"table {tableName} already exists")

table ArtistTracker created


In [7]:
# test the unique contraint

artistTb = meta.tables.get(tableName)

try:
    SQL.insert(artistTb).values({'collectionId': 'test 1', 'deviantId': 'test 2'}).execute()
except Exception as ex:
    print(ex)

Unconsumed column names: deviantArtId


In [8]:
try:
    SQL.insert(artistTb).values({'collectionId': 'test 3'}).execute()
except Exception as ex:
    print(ex)

In [9]:
try:
    SQL.insert(artistTb).values({'collectionId': 'test 3'}).execute()
except Exception as ex:
    print(ex)

(sqlite3.IntegrityError) UNIQUE constraint failed: ArtistTracker.collectionId, ArtistTracker.deviantId
[SQL: INSERT INTO "ArtistTracker" ("collectionId") VALUES (?)]
[parameters: ('test 3',)]
(Background on this error at: http://sqlalche.me/e/14/gkpj)


In [10]:
statement = SQL.select(artistTb)
results = statement.execute().fetchall()
print(*results[:5], sep='\n')

(1, 'test 3', 'no-id', None, 'to-do', False, None, None)


In [11]:
SQL.delete(artistTb).where(artistTb.c.collectionId=='test 1').execute()
SQL.delete(artistTb).where(artistTb.c.collectionId=='test 3').execute()

statement = SQL.select(artistTb)
results = statement.execute().fetchall()
print(f"rows in table: {len(results)}")

rows in table: 0


In [12]:
# delete the table when needed
# delete is not reflected in the meta

# deleteTable = meta.tables.get(tableName)

# if deleteTable is not None:
#     try:
#         deleteTable.drop(engine)
#         print("table deleted")
#     except Exception as ex:
#         print(ex)